# Инструкция

Соберите случайный ансамбль из нескольких методов выделения факторов - корреляции, взаимной информации, важности признаков, главных компонент, независимых компонент и др. Получите не менее 3 наборов из 5 наиболее важных признаков.

Соберите для каждого набора ансамбль стекинга для задачи, используя (но не ограничиваясь) решающими деревьями, CatBoost, линейной регрессией - всего не менее 3 ансамблей стекинга, каждый из которых состоит из большого числа разнородных моделей.

Используя эти ансамбли, предскажите продолжительность жизни на 2019 год.

\* Дополнительно: найдите значения факторов Росстата за 2019 год и предскажите продолжительность жизни на 2020 год.

Данные:
video.ittensive.com/machine-learning/sc-tatar2020/rosstat/rosstat.csv

Итоговый файл с кодом (.py или .ipynb) выложите в github с портфолио.

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import ExtraTreesRegressor, RandomForestRegressor
from sklearn.feature_selection import mutual_info_regression
from sklearn.decomposition import PCA, FastICA
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
import warnings
warnings.simplefilter('ignore')


def linear_extrapolation(x):
    X = np.array(x.dropna().index.astype(int)).reshape(-1, 1)
    Y = np.array(x.dropna().values).reshape(-1, 1)
    if X.shape[0] > 0:
        f = LinearRegression().fit(X, Y)
        for i in x.index:
            v = x.loc[i]
            if v != v:
                v = f.predict([[int(i)]])[0][0]
                if v < 0:
                    v = 0
                x.loc[i] = v
    return x


data = pd.read_csv(
    "https://video.ittensive.com/machine-learning/sc-tatar2020/rosstat/rosstat.csv",
    na_values=["-", " - ","...","…"," -"]
)

features = data["feature"]
data.drop("feature", inplace=True, axis=1)
data.interpolate(method="linear", axis=1, inplace=True)

data = data.apply(linear_extrapolation, axis=1)

data["feature"] = features
data.dropna(inplace=True)
features = data["feature"]

data = data.T[:len(data.columns) - 1].astype("float")
data.drop("2019", inplace=True)

y_columns = [
    "ОЖИДАЕМАЯ ПРОДОЛЖИТЕЛЬНОСТЬ ЖИЗНИ ПРИ РОЖДЕНИИ 1.16.1. Все население (число лет)",
    "ОЖИДАЕМАЯ ПРОДОЛЖИТЕЛЬНОСТЬ ЖИЗНИ ПРИ РОЖДЕНИИ 1.16.2. Мужчины (число лет)",
    "ОЖИДАЕМАЯ ПРОДОЛЖИТЕЛЬНОСТЬ ЖИЗНИ ПРИ РОЖДЕНИИ 1.16.3. Женщины (число лет)"
]

data.columns = features
y = data[y_columns[0]]
x = data.drop(y_columns, axis=1)

scaler = MinMaxScaler()
x = pd.DataFrame(scaler.fit_transform(x), columns=x.columns)
features = x.columns
len(features)

474

In [2]:
feature_indices = dict()

## Определение наборов признаков

### Корреляция данных

In [3]:
def corr_ensemble_score(feature_values, y):
    result_score = 0
    scores = np.zeros(7)
    scores[0] = LinearRegression().fit(x_feature, y).score(x_feature, y)
    scores[1] = Ridge(alpha=.1).fit(x_feature, y).score(x_feature, y)
    scores[2] = Ridge(alpha=.3).fit(x_feature, y).score(x_feature, y)
    scores[3] = Ridge(alpha=.8).fit(x_feature, y).score(x_feature, y)
    scores[4] = Lasso(alpha=.1).fit(x_feature, y).score(x_feature, y)
    scores[5] = Lasso(alpha=.3).fit(x_feature, y).score(x_feature, y)
    scores[6] = Lasso(alpha=.8).fit(x_feature, y).score(x_feature, y)
    for score in scores:
        if score > 0.96:
            result_score += score
    return result_score


scores = np.zeros(len(features))
for feature_index, feature in enumerate(features):
    x_feature = np.array(x[feature]).reshape(-1, 1)
    feature_score = corr_ensemble_score(x_feature, y)
    scores[feature_index] += (feature_score - 0.96)

feature_indices["corr"] = np.argsort(scores)[::-1][:5]
print(*features[feature_indices["corr"]], sep='\n')

20.5. ИСПОЛНЕНИЕ БЮДЖЕТА ФОНДА СОЦИАЛЬНОГО СТРАХОВАНИЯ РОССИЙСКОЙ ФЕДЕРАЦИИ 20.5.2. Расходование (миллионов рублей)
12.12. ИТОГИ ВЫБОРОЧНЫХ ОБСЛЕДОВАНИЙ 12.12.3. Объем выручки от продажи товаров, продукции, работ, услуг
13.2. ОБЪЕМ ОТГРУЖЕННЫХ ТОВАРОВ СОБСТВЕННОГО ПРОИЗВОДСТВА, ВЫПОЛНЕННЫХ РАБОТ И УСЛУГ СОБСТВЕННЫМИ СИЛАМИ ПО ВИДАМ ЭКОНОМИЧЕСКОЙ ДЕЯТЕЛЬНОСТИ 13.2.6. «Обеспечение электрической энергией, газом и паром; кондиционирование воздуха» в соответствии с ОКВЭД2 (в фактически действовавших ценах; миллионов рублей)
ДЕНЕЖНЫЕ ДОХОДЫ НАСЕЛЕНИЯ 3.5. МЕДИАННАЯ ЗАРАБОТНАЯ ПЛАТА РАБОТНИКОВ ОРГАНИЗАЦИЙ (по данным выборочного обследования; рублей)
19.6. ВНУТРЕННИЕ ТЕКУЩИЕ ЗАТРАТЫ НА НАУЧНЫЕ ИССЛЕДОВАНИЯ И РАЗРАБОТКИ ПО ВИДАМ ЗАТРАТ   19.6.2. Оплата труда  (миллионов рублей)


### Взаимная информация

In [4]:
scores = mutual_info_regression(x, y)
scores /= np.max(scores)
scores -= .85
feature_indices["mi"] = np.argsort(scores)[::-1][:5]
print(*features[feature_indices["mi"]], sep='\n')

12.12. ИТОГИ ВЫБОРОЧНЫХ ОБСЛЕДОВАНИЙ 12.12.3. Объем выручки от продажи товаров, продукции, работ, услуг
18.7. ИСПОЛЬЗОВАНИЕ СЕТИ ИНТЕРНЕТ НАСЕЛЕНИЕМ 18.7.2. Население, использовавшее сеть Интернет каждый день или почти каждый день2) (по данным выборочного обследования населения по вопросам использования ИКТ; в процентах от общей численности населения соответствующего субъекта Российской Федерации)
13.2. ОБЪЕМ ОТГРУЖЕННЫХ ТОВАРОВ СОБСТВЕННОГО ПРОИЗВОДСТВА, ВЫПОЛНЕННЫХ РАБОТ И УСЛУГ СОБСТВЕННЫМИ СИЛАМИ ПО ВИДАМ ЭКОНОМИЧЕСКОЙ ДЕЯТЕЛЬНОСТИ 13.2.6. «Обеспечение электрической энергией, газом и паром; кондиционирование воздуха» в соответствии с ОКВЭД2 (в фактически действовавших ценах; миллионов рублей)
18.7. ИСПОЛЬЗОВАНИЕ СЕТИ ИНТЕРНЕТ НАСЕЛЕНИЕМ 18.7.1.  Население, использовавшее сеть Интернет2) (по данным выборочного обследования населения по вопросам использования ИКТ; в процентах от общей численности населения соответствующего субъекта Российской Федерации)
12.8. ДЕБИТОРСКАЯ ЗАДОЛЖЕННОСТ

### Важность признаков

In [5]:
%%time
scores = np.zeros(len(features))
for i in range(10, 100):
    scores += RandomForestRegressor(n_estimators=i, random_state=i).fit(x, y).feature_importances_
    scores += ExtraTreesRegressor(n_estimators=i, random_state=i).fit(x, y).feature_importances_
    
feature_indices["importance"] = np.argsort(scores)[::-1][:5]
print(*features[feature_indices["importance"]], sep='\n')

ЖИЛИЩНЫЕ УСЛОВИЯ НАСЕЛЕНИЯ 3.26. УДЕЛЬНЫЙ ВЕС АВАРИЙНОГО ЖИЛИЩНОГО ФОНДА В ОБЩЕЙ ПЛОЩАДИ ВСЕГО ЖИЛИЩНОГО ФОНДА (в процентах)
ЗАБОЛЕВАЕМОСТЬ на 1000 человек населения ПО ОСНОВНЫМ КЛАССАМ БОЛЕЗНЕЙ;2) 5.9.9. Болезни органов дыхания (зарегистрировано заболеваний у пациентов с диагнозом, установленным впервые в жизни)
4.5. ЧИСЛЕННОСТЬ УЧИТЕЛЕЙ ОРГАНИЗАЦИЙ, ОСУЩЕСТВЛЯЮЩИХ ОБРАЗОВАТЕЛЬНУЮ ДЕЯТЕЛЬНОСТЬ ПО ОБРАЗОВАТЕЛЬНЫМ ПРОГРАММАМ НАЧАЛЬНОГО, ОСНОВНОГО И СРЕДНЕГО ОБЩЕГО ОБРАЗОВАНИЯ; 2) (на 20 сентября; тысяч человек)
СТРОИТЕЛЬНАЯ ДЕЯТЕЛЬНОСТЬ 15.1. ЧИСЛО ДЕЙСТВУЮЩИХ СТРОИТЕЛЬНЫХ ОРГАНИЗАЦИЙ  (на конец года)
18.1. ИСПОЛЬЗОВАНИЕ ИНФОРМАЦИОННЫХ И КОММУНИКАЦИОННЫХ ТЕХНОЛОГИЙ В ОРГАНИЗАЦИЯХ  18.1.4. Организации, использовавшие глобальные информационные сети (в процентах от общего числа обследованных организаций  соответствующего субъекта Российской федерации)
CPU times: total: 12.7 s
Wall time: 12.6 s


### Метод главных компонент

In [6]:
scores = np.zeros(len(features))
for n_components in range(2, 20):
    scores += np.apply_along_axis(
        lambda cortege: np.abs(cortege).sum(),
        0,
        PCA(n_components=n_components, random_state=n_components).fit(x).components_
    )

feature_indices["pca"] = np.argsort(scores)[::-1][:5]
print(*features[feature_indices["pca"]], sep='\n')

4.11. ЧИСЛО ФИЛИАЛОВ ОБРАЗОВАТЕЛЬНЫХ ОРГАНИЗАЦИЙ, ОСУЩЕСТВЛЯЮЩИХ ОБРАЗОВАТЕЛЬНУЮ ДЕЯТЕЛЬНОСТЬ ПО ОБРАЗОВАТЕЛЬНЫМ ПРОГРАММАМ СРЕДНЕГО ПРОФЕССИОНАЛЬНОГО ОБРАЗОВАНИЯ (на начало учебного года)
ДЕНЕЖНЫЕ ДОХОДЫ НАСЕЛЕНИЯ 3.10. СТРУКТУРА СОЦИАЛЬНЫХ ВЫПЛАТ;2);3) (в процентах)
ДЕНЕЖНЫЕ ДОХОДЫ НАСЕЛЕНИЯ 3.16. СТРУКТУРА ПОТРЕБИТЕЛЬСКИХ РАСХОДОВ ДОМАШНИХ ХОЗЯЙСТВ (по итогам выборочного обследования бюджетов домашних хозяйств; в процентах)
ДЕНЕЖНЫЕ ДОХОДЫ НАСЕЛЕНИЯ 3.15. СТРУКТУРА ИСПОЛЬЗОВАНИЯ ДЕНЕЖНЫХ ДОХОДОВ НАСЕЛЕНИЯ ;2) (в процентах от общего объема денежных доходов)
ДЕНЕЖНЫЕ ДОХОДЫ НАСЕЛЕНИЯ 3.9. СТРУКТУРА ДЕНЕЖНЫХ ДОХОДОВ НАСЕЛЕНИЯ;2) (в процентах от общего объема денежных доходов)


### Анализ независимых компонент

In [7]:
scores = np.zeros(len(features))
for n_components in range(2, 20):
    scores += np.apply_along_axis(
        lambda cortege: np.abs(cortege).sum(),
        0,
        FastICA(n_components=n_components, random_state=n_components).fit(x).components_
    )

feature_indices["ica"] = np.argsort(scores)[::-1][:5]
print(*features[feature_indices["ica"]], sep='\n')

ОБЩАЯ ХАРАКТЕРИСТИКА ПРЕДПРИЯТИЙ И ОРГАНИЗАЦИЙ 12.1. ЧИСЛО ПРЕДПРИЯТИЙ И ОРГАНИЗАЦИЙ  (на конец года)
9.3. ИНДЕКС ФИЗИЧЕСКОГО ОБЪЕМА ВАЛОВОГО РЕГИОНАЛЬНОГО ПРОДУКТА (в постоянных ценах; в процентах к предыдущему году)
4.25. ЧИСЛЕННОСТЬ СТУДЕНТОВ, ОБУЧАЮЩИХСЯ ПО ПРОГРАММАМ БАКАЛАВРИАТА, СПЕЦИАЛИТЕТА, МАГИСТРАТУРЫ на 10 000 человек населения (на начало учебного года; человек)
НАГРУЗКА НА РАБОТНИКОВ СФЕРЫ ЗДРАВООХРАНЕНИЯ 5.5.2. Численность населения на одного работника  среднего медицинского персонала (на конец года; человек)
21.10. ИНДЕКСЫ ЦЕН ПРОИЗВОДИТЕЛЕЙ ПРОМЫШЛЕННЫХ ТОВАРОВ ПО ВИДАМ ЭКОНОМИЧЕСКОЙ ДЕЯТЕЛЬНОСТИ 21.10.5. Обрабатывающие производства в соответствии с ОКВЭД в 2000-2016 гг.


### Случайный набор признаков
Выберем по 50 признаков случайным образом.

Обучим несколько моделей на случайном поднаборе признаков, отранжируем признаки по значимости, возьмем 10 наиболее значащих признаков, повторим такой выбор несколько раз, затем выберем топ5 из нескольких итераций

In [8]:
%%time
subset_size = 50
iterations = 5000
index_weights = np.array([weight for weight in range(10, 0, -1)])
model_weights = [1, 1, 1, 1, 1, 1]


def make_ensemble(x, y):
    return {
        "ridge1": np.argsort(np.abs(Ridge(alpha=.1).fit(x, y).coef_))[::-1],
        "ridge3": np.argsort(np.abs(Ridge(alpha=.3).fit(x, y).coef_))[::-1],
        "ridge8": np.argsort(np.abs(Ridge(alpha=.8).fit(x, y).coef_))[::-1],
        "lasso1": np.argsort(np.abs(Lasso(alpha=.1).fit(x, y).coef_))[::-1],
        "lasso3": np.argsort(np.abs(Lasso(alpha=.3).fit(x, y).coef_))[::-1],
        "lasso8": np.argsort(np.abs(Lasso(alpha=.8).fit(x, y).coef_))[::-1],
    }


def indices_of_random_features(number_of_features, max_index=len(features)):
    return np.random.randint(max_index, size=number_of_features)


scores = np.zeros(len(features))
for iteration in range(iterations):
    ensemble = make_ensemble(x[features[indices_of_random_features(subset_size)]], y)
    model_index = 0
    for _, model_coefs in ensemble.items():
        siginificant_feature_indices = np.argsort(np.abs(model_coefs))[::-1][:10]
        scores[siginificant_feature_indices] += (index_weights * model_weights[model_index])
        model_index += 1
        
feature_indices["random"] = np.argsort(scores)[::-1][:5]
print(*features[feature_indices["random"]], sep='\n')

17.4. ПЕРЕВОЗКИ ГРУЗОВ АВТОМОБИЛЬНЫМ  ТРАНСПОРТОМ ОРГАНИЗАЦИЙ ВСЕХ ВИДОВ ДЕЯТЕЛЬНОСТИ (миллионов тонн)
17.5.  ГРУЗООБОРОТ АВТОМОБИЛЬНОГО ТРАНСПОРТА ОРГАНИЗАЦИЙ ВСЕХ ВИДОВ ДЕЯТЕЛЬНОСТИ (миллионов тонно-километров)
17.3. ПЛОТНОСТЬ ЖЕЛЕЗНОДОРОЖНЫХ ПУТЕЙ ОБЩЕГО ПОЛЬЗОВАНИЯ  (на конец года; км путей на 10000 км2 территории)
17.6. ПЕРЕВОЗКИ ПАССАЖИРОВ АВТОБУСАМИ ОБЩЕГО ПОЛЬЗОВАНИЯ (миллионов человек)
17.7. ПАССАЖИРООБОРОТ АВТОБУСОВ ОБЩЕГО ПОЛЬЗОВАНИЯ  (миллионов пассажиро-километров)
CPU times: total: 36.5 s
Wall time: 36.5 s


## Ансамбль

Сдвинем факторы на год назад: чтобы предсказывать по значениям текущего года - следующий

In [9]:
x_ = x[:-1]
y_ = np.array(y[1:])

### Для каждого набора данных строится:
- 100 моделей RandomForestRegressor
- 100 моделей ExtraTreesRegressor
- 100 моделей XGBRegressor
- 100 моделей CatBoostRegressor
- 300 моделей Lasso
- 300 моделей Ridge

In [10]:
MODELS_IN_EACH_TYPE = 100
ensemble = dict()
for set_name, set_indices in feature_indices.items():
    ensemble[set_name] = list()
    current_x = x_.iloc[:, set_indices]
    for i in range(MODELS_IN_EACH_TYPE):
        ensemble[set_name].append(RandomForestRegressor(n_estimators=30 + i, random_state=i).fit(current_x, y_))
        ensemble[set_name].append(ExtraTreesRegressor(n_estimators=30 + i, random_state=i).fit(current_x, y_))
        ensemble[set_name].append(XGBRegressor(max_depth=16, n_estimators=30+i, random_state=i).fit(current_x, y_))
        ensemble[set_name].append(CatBoostRegressor(iterations=20, depth=16, random_state=i, verbose=False).fit(current_x, y_))
        ensemble[set_name].append(Lasso(alpha=.1, random_state=i).fit(current_x, y_))
        ensemble[set_name].append(Lasso(alpha=.3, random_state=i).fit(current_x, y_))
        ensemble[set_name].append(Lasso(alpha=.8, random_state=i).fit(current_x, y_))
        ensemble[set_name].append(Ridge(alpha=.1, random_state=i).fit(current_x, y_))
        ensemble[set_name].append(Ridge(alpha=.3, random_state=i).fit(current_x, y_))
        ensemble[set_name].append(Ridge(alpha=.8, random_state=i).fit(current_x, y_))        

print("Количество моделей в ансамбле:", len(ensemble) * MODELS_IN_EACH_TYPE * 10)

Количество моделей в ансамбле: 6000


### Формирование предсказания на 2019 год

In [11]:
prediction = 0
x_2019 = x[-1:]
for set_name, set_indices in feature_indices.items():
    current_x = x_2019.iloc[:, set_indices]
    for model_index in range(len(ensemble[set_name])):
        prediction += ensemble[set_name][model_index].predict(current_x)

prediction /= (len(ensemble) * MODELS_IN_EACH_TYPE * 10)

print(
    "Прогноз ожидаемой продолжительности жизни в 2019 год составляет",
    round(prediction[0], 1),
    "лет"
)

Прогноз ожидаемой продолжительности жизни в 2019 год составляет 73.0 лет


#### *Дополнительно: найдите значения факторов Росстата за 2019 год и предскажите продолжительность жизни на 2020 год.

In [12]:
data_2020 = pd.read_csv("rosstat_2019.csv", usecols=data.columns)
y_2020 = data_2020[y_columns[0]]
x_2020 = data_2020.drop(y_columns, axis=1)

scaler_2020 = MinMaxScaler()
x_2020 = pd.DataFrame(scaler.fit_transform(x_2020), columns=x_2020.columns)
features = x_2020.columns

x_ = x_2020[:-1]
y_ = np.array(y_2020[1:])

In [13]:
ensemble_2020 = dict()
for set_name, set_indices in feature_indices.items():
    ensemble_2020[set_name] = list()
    current_x = x_.iloc[:, set_indices]
    for i in range(MODELS_IN_EACH_TYPE):
        ensemble_2020[set_name].append(RandomForestRegressor(n_estimators=30 + i, random_state=i).fit(current_x, y_))
        ensemble_2020[set_name].append(ExtraTreesRegressor(n_estimators=30 + i, random_state=i).fit(current_x, y_))
        ensemble_2020[set_name].append(XGBRegressor(max_depth=16, n_estimators=30+i, random_state=i).fit(current_x, y_))
        ensemble_2020[set_name].append(CatBoostRegressor(iterations=20, depth=16, random_state=i, verbose=False).fit(current_x, y_))
        ensemble_2020[set_name].append(Lasso(alpha=.1, random_state=i).fit(current_x, y_))
        ensemble_2020[set_name].append(Lasso(alpha=.3, random_state=i).fit(current_x, y_))
        ensemble_2020[set_name].append(Lasso(alpha=.8, random_state=i).fit(current_x, y_))
        ensemble_2020[set_name].append(Ridge(alpha=.1, random_state=i).fit(current_x, y_))
        ensemble_2020[set_name].append(Ridge(alpha=.3, random_state=i).fit(current_x, y_))
        ensemble_2020[set_name].append(Ridge(alpha=.8, random_state=i).fit(current_x, y_))        

print("Количество моделей в ансамбле:", len(ensemble_2020) * MODELS_IN_EACH_TYPE * 10)

Количество моделей в ансамбле: 6000


In [14]:
prediction_2020 = 0
x_2020 = x_2019[-1:]
for set_name, set_indices in feature_indices.items():
    current_x = x_2020.iloc[:, set_indices]
    for model_index in range(len(ensemble[set_name])):
        prediction_2020 += ensemble_2020[set_name][model_index].predict(current_x)

prediction_2020 /= (len(ensemble_2020) * MODELS_IN_EACH_TYPE * 10)

print(
    "Прогноз ожидаемой продолжительности жизни в 2020 год составляет",
    round(prediction_2020[0], 1),
    "лет"
)

Прогноз ожидаемой продолжительности жизни в 2020 год составляет 73.7 лет
